## IMPORTS

In [2]:
from itertools import combinations, compress, combinations_with_replacement
import numpy as np

## TESTING

In [11]:
numbers = [5, 6, 7, 9]
combs = list(combinations(set(numbers), 2))

print(combs)

combs = list(combinations(numbers, 2))
print(combs)

combs = list(set(combinations(numbers, 2)))
print(combs)

[(9, 5), (9, 6), (9, 7), (5, 6), (5, 7), (6, 7)]
[(5, 6), (5, 7), (5, 9), (6, 7), (6, 9), (7, 9)]
[(5, 7), (7, 9), (6, 7), (5, 6), (5, 9), (6, 9)]


## FUNCTIONS

In [48]:
def prepare_numbers(numbers):
    for i, number in enumerate(numbers):
        numbers[i] = (number, str(number))

    return numbers

def apply_operation(numero_1, numero_2, operation):
    """
        Input:
            - numero_1: tuple of value and equation - (int, str)
            - numero_2: tuple of value and equation - (int, str)
            - operations: operation to apply
        Output:
            - tuple of the resulting value and equation of the operation - (int, str)  
    """
    if operation == "*":
        return (numero_1[0]*numero_2[0], f"({numero_1[1]} * {numero_2[1]})")
    elif operation == "÷":
        return (numero_1[0]/numero_2[0], f"({numero_1[1]} ÷ {numero_2[1]})")
    elif operation == "+":
        return (numero_1[0]+numero_2[0], f"({numero_1[1]} + {numero_2[1]})")
    elif operation == "-":
        return (numero_1[0]-numero_2[0], f"({numero_1[1]} - {numero_2[1]})")
    elif operation == "&":
        temp = numero_1[1]+numero_2[1] 
        return (int(temp), temp)
    else:
        raise NotImplementedError(f"oOperation {operation} not supported.")
    
## Alternative implementation:
# def apply_operation(numero_1, numero_2, operation):
#     """
#         Input:
#             - numero_1: tuple of value and equation - (int, str)
#             - numero_2: tuple of value and equation - (int, str)
#             - operations: operation to apply
#         Output:
#             - tuple of the resulting value and equation of the operation - (int, str)  
#     """
#     match operation:
#         case "*":
#             return (numero_1[0]*numero_2[0], f"({numero_1[1]} * {numero_2[1]})")
#         case "÷":
#             return (numero_1[0]/numero_2[0], f"({numero_1[1]} ÷ {numero_2[1]})")
#         case "+":
#             return (numero_1[0]+numero_2[0], f"({numero_1[1]} + {numero_2[1]})")
#         case "-":
#             return (numero_1[0]-numero_2[0], f"({numero_1[1]} - {numero_2[1]})")
#         case "&":
#             temp = numero_1[1]+numero_2[1] 
#             return (int(temp), temp)

In [49]:
def recursive_all_ten_v1(results, numbers, operations=["+", "-", "÷", "*", "&"], counting_only=False):
    # Has problems with duplicate solutions in 2 cases:
    #   1. When two numbers are the same the operations -, ÷ and & also become commutative
    #   2. When we have the case of f(g(a, b), h(c, d)) it is the same solution for any order of h() and g().

    # This function can still be used for the non_valid_starts experiment since we are only interested in
    # cases where there are no solutions so duplicates will not cause any problems.
    """
        Input:
            - results:          dict for storing the results - {1: [], 2:[], ..., 10:[]}. You can define the target space
                                with different initialisation, since the algorithm will save the resulting equations for all
                                keys in the dictionary. 
                                (If counting_only == True results should be of the form {1: 0, ..., 10: 0})
            - numbers:          list of tuples - [(value, "equation"), ...] of type [(int, str), ...]
            - operations:       list of possible operations between numbers
            - counting_only:    boolean indicator for a later experiment that makes the function only store
                                the number of solutions per target (!! the results need to be initiated differently !!)
        Output:
            - results:          dict of all possible equations for given goal numbers between 1-10
                                with goal numbers as keys and list of equations as values - 
                                {1: ["equation1", ...], 2:["equation1", ...], ..., 10:["equation1", ...]}
                                If counting_only == True then results will be of form {1: n_1, ..., 10: n_10} where
                                n_i corresponds to the number of solution for target i.
    """

    if len(numbers) == 1:
        temp_value = numbers[0][0]
        temp_equation = numbers[0][1]
        if abs(temp_value - int(temp_value) < 0.00005) and int(temp_value)<=max(results.keys()) and int(temp_value)>=min(results.keys()):
            # we found a valid solution
            if counting_only:
                results[int(temp_value)]+=1
            else:
                results[int(temp_value)].append(temp_equation)

    else:
        combs = list(set(combinations(numbers, 2)))
        print(numbers)

        for option in combs:
            numbers_copy = numbers.copy()
            numbers_copy.remove(option[0])
            numbers_copy.remove(option[1])
            for operation in operations:
                if operation in ["+", "*"]:
                    # these are the only commutative operations, so no matter the order
                    # the equations will be interchangable
                    temp = apply_operation(option[0], option[1], operation)
                    recursive_all_ten_v1(results, numbers_copy + [temp], counting_only=counting_only)

                elif operation == "-":
                    temp_1 = apply_operation(option[0], option[1], operation)
                    recursive_all_ten_v1(results, numbers_copy + [temp_1], counting_only=counting_only)
                    temp_2 = apply_operation(option[1], option[0], operation)
                    recursive_all_ten_v1(results, numbers_copy + [temp_2], counting_only=counting_only)

                elif operation == "÷":
                    # in the case of the ÷ operator we have to prevent division by zero
                    if option[0][0] != 0 and option[1][0] != 0:
                        temp_1 = apply_operation(option[0], option[1], operation)
                        recursive_all_ten_v1(results, numbers_copy + [temp_1], counting_only=counting_only)
                        temp_2 = apply_operation(option[1], option[0], operation)
                        recursive_all_ten_v1(results, numbers_copy + [temp_2], counting_only=counting_only)
                    elif option[0][0] == 0 and option[1][0] != 0:
                        temp = apply_operation(option[0], option[1], operation)
                        recursive_all_ten_v1(results, numbers_copy + [temp], counting_only=counting_only)
                    elif option[0][0] != 0 and option[1][0] == 0:
                        temp = apply_operation(option[1], option[0], operation)
                        recursive_all_ten_v1(results, numbers_copy + [temp], counting_only=counting_only)
                    else:
                        pass

                else:
                    # in the case of & opeartion we are only allowed to concatenate original numbers
                    ## TO-DO: improve this to maybe allow 3 digits to get concatened (in the original
                    ## game the only allowed numbers are 1-9 so this would not result in any valid 
                    ## equiation but would be interesting to try.)
                    if len(option[0][1]) > 1 or len(option[1][1]) > 1:
                        pass
                    else:
                        temp_1 = apply_operation(option[0], option[1], operation)
                        recursive_all_ten_v1(results, numbers_copy + [temp_1], counting_only=counting_only)
                        temp_2 = apply_operation(option[1], option[0], operation)
                        recursive_all_ten_v1(results, numbers_copy + [temp_2], counting_only=counting_only)

    return results


def recursive_all_ten_v2(results, numbers, operations=["+", "-", "÷", "*", "&"], counting_only=False):
    # Deals with the problem of having two equal numbers input into functions -, ÷ and &, but still has
    # problems with the case of f(g(a, b), h(c, d)) since it is the same solution for any order of h() and g().

    # This function can still be used for the non_valid_starts experiment since we are only interested in
    # cases where there are no solutions so duplicates will not cause any problems.
    """
        Input:
            - results:          dict for storing the results - {1: [], 2:[], ..., 10:[]}. You can define the target space
                                with different initialisation, since the algorithm will save the resulting equations for all
                                keys in the dictionary. 
                                (If counting_only == True results should be of the form {1: 0, ..., 10: 0})
            - numbers:          list of tuples - [(value, "equation"), ...] of type [(int, str), ...]
            - operations:       list of possible operations between numbers
            - counting_only:    boolean indicator for a later experiment that makes the function only store
                                the number of solutions per target (!! the results need to be initiated differently !!)
        Output:
            - results:          dict of all possible equations for given goal numbers between 1-10
                                with goal numbers as keys and list of equations as values - 
                                {1: ["equation1", ...], 2:["equation1", ...], ..., 10:["equation1", ...]}
                                If counting_only == True then results will be of form {1: n_1, ..., 10: n_10} where
                                n_i corresponds to the number of solution for target i.
    """

    if len(numbers) == 1:
        temp_value = numbers[0][0]
        temp_equation = numbers[0][1]
        if abs(temp_value - int(temp_value) < 0.00005) and int(temp_value)<=max(results.keys()) and int(temp_value)>=min(results.keys()):
            # we found a valid solution
            if counting_only:
                results[int(temp_value)]+=1
            else:
                results[int(temp_value)].append(temp_equation)

    else:
        combs = list(set(combinations(numbers, 2)))

        for option in combs:
            numbers_copy = numbers.copy()
            numbers_copy.remove(option[0])
            numbers_copy.remove(option[1])
            for operation in operations:
                if operation in ["+", "*"]:
                    # these are the only commutative operations, so no matter the order
                    # the equations will be interchangable
                    temp = apply_operation(option[0], option[1], operation)
                    recursive_all_ten_v2(results, numbers_copy + [temp], counting_only=counting_only)

                elif operation == "-":
                    if option[0][0] == option[0][1]:
                        # if both values are the same then this also becomes commutative
                        temp = apply_operation(option[0], option[1], operation)
                        recursive_all_ten_v2(results, numbers_copy + [temp], counting_only=counting_only)
                    else:
                        temp_1 = apply_operation(option[0], option[1], operation)
                        recursive_all_ten_v2(results, numbers_copy + [temp_1], counting_only=counting_only)
                        temp_2 = apply_operation(option[1], option[0], operation)
                        recursive_all_ten_v2(results, numbers_copy + [temp_2], counting_only=counting_only)

                elif operation == "÷":
                    # in the case of the ÷ operator we have to prevent division by zero
                    if option[0][0] == 0 and option[1][0] == 0:
                        pass
                    elif option[0][0] == 0 and option[1][0] != 0:
                        temp = apply_operation(option[0], option[1], operation)
                        recursive_all_ten_v2(results, numbers_copy + [temp], counting_only=counting_only)
                    elif option[0][0] != 0 and option[1][0] == 0:
                        temp = apply_operation(option[1], option[0], operation)
                        recursive_all_ten_v2(results, numbers_copy + [temp], counting_only=counting_only)
                    elif option[0][0] == option[1][0]:
                        # if both values are the same then this also becomes commutative
                        temp = apply_operation(option[0], option[1], operation)
                        recursive_all_ten_v2(results, numbers_copy + [temp], counting_only=counting_only)
                    else:
                        temp_1 = apply_operation(option[0], option[1], operation)
                        recursive_all_ten_v2(results, numbers_copy + [temp_1], counting_only=counting_only)
                        temp_2 = apply_operation(option[1], option[0], operation)
                        recursive_all_ten_v2(results, numbers_copy + [temp_2], counting_only=counting_only)

                else:
                    # in the case of & opeartion we are only allowed to concatenate original numbers
                    ## TO-DO: improve this to maybe allow 3 digits to get concatened (in the original
                    ## game the only allowed numbers are 1-9 so this would not result in any valid 
                    ## equiation but would be interesting to try.)
                    if len(option[0][1]) > 1 or len(option[1][1]) > 1:
                        pass
                    elif option[0][0] == option[0][1]:
                        # if both values are the same then this also becomes commutative
                        temp = apply_operation(option[0], option[1], operation)
                        recursive_all_ten_v2(results, numbers_copy + [temp], counting_only=counting_only)
                    else:
                        temp_1 = apply_operation(option[0], option[1], operation)
                        recursive_all_ten_v2(results, numbers_copy + [temp_1], counting_only=counting_only)
                        temp_2 = apply_operation(option[1], option[0], operation)
                        recursive_all_ten_v2(results, numbers_copy + [temp_2], counting_only=counting_only)

    return results


def recursive_all_ten(results, numbers, operations=["+", "-", "÷", "*", "&"], counting_only=False):
    """
        Input:
            - results:          dict for storing the results - {1: [], 2:[], ..., 10:[]}. You can define the target space
                                with different initialisation, since the algorithm will save the resulting equations for all
                                keys in the dictionary. 
                                (If counting_only == True results should be of the form {1: 0, ..., 10: 0})
            - numbers:          list of tuples - [(value, "equation"), ...] of type [(int, str), ...]
            - operations:       list of possible operations between numbers
            - counting_only:    boolean indicator for a later experiment that makes the function only store
                                the number of solutions per target (!! the results need to be initiated differently !!)
        Output:
            - results:          dict of all possible equations for given goal numbers between 1-10
                                with goal numbers as keys and list of equations as values - 
                                {1: ["equation1", ...], 2:["equation1", ...], ..., 10:["equation1", ...]}
                                If counting_only == True then results will be of form {1: n_1, ..., 10: n_10} where
                                n_i corresponds to the number of solution for target i.
    """

    if len(numbers) == 1:
        temp_value = numbers[0][0]
        temp_equation = numbers[0][1]
        if abs(temp_value - round(temp_value)) < 0.00005 and round(temp_value)<=max(results.keys()) and round(temp_value)>=min(results.keys()):
            # we found a valid solution
            # if counting_only:
            #     results[int(temp_value)]+=1
            # else:
            #     results[int(temp_value)].append(temp_equation)
            if temp_equation in results[int(round(temp_value))]:
                pass
                # An edge case that is not dealt with in the main code is when the first operation (op_1) is used on 2 starting 
                # numbers (let's call them a and b) and the second (op_2) is used on the remaining 2 (c and d). This produces the 
                # same equation as performing the op_2 on c and d first and then op_1 on a and b. That is the reson for this if
                # satement, although it messed up the counting_only functionality which. TO-DO: maybe possible to check this beforehand?
            else:
                results[int(round(temp_value))].append(temp_equation)


    else:
        combs = list(set(combinations(numbers, 2)))

        for option in combs:
            numbers_copy = numbers.copy()
            numbers_copy.remove(option[0])
            numbers_copy.remove(option[1])
            for operation in operations:
                if operation in ["+", "*"]:
                    # these are the only commutative operations, so no matter the order
                    # the equations will be interchangable
                    temp = apply_operation(option[0], option[1], operation)
                    recursive_all_ten(results, numbers_copy + [temp], counting_only=counting_only)

                elif operation == "-":
                    if option[0][0] == option[0][1]:
                        # if both values are the same then this also becomes commutative
                        temp = apply_operation(option[0], option[1], operation)
                        recursive_all_ten(results, numbers_copy + [temp], counting_only=counting_only)
                    else:
                        temp_1 = apply_operation(option[0], option[1], operation)
                        recursive_all_ten(results, numbers_copy + [temp_1], counting_only=counting_only)
                        temp_2 = apply_operation(option[1], option[0], operation)
                        recursive_all_ten(results, numbers_copy + [temp_2], counting_only=counting_only)

                elif operation == "÷":
                    # in the case of the ÷ operator we have to prevent division by zero
                    if option[0][0] == 0 and option[1][0] == 0:
                        pass
                    elif option[0][0] == 0 and option[1][0] != 0:
                        temp = apply_operation(option[0], option[1], operation)
                        recursive_all_ten(results, numbers_copy + [temp], counting_only=counting_only)
                    elif option[0][0] != 0 and option[1][0] == 0:
                        temp = apply_operation(option[1], option[0], operation)
                        recursive_all_ten(results, numbers_copy + [temp], counting_only=counting_only)
                    elif option[0][0] == option[1][0]:
                        # if both values are the same then this also becomes commutative
                        temp = apply_operation(option[0], option[1], operation)
                        recursive_all_ten(results, numbers_copy + [temp], counting_only=counting_only)
                    else:
                        temp_1 = apply_operation(option[0], option[1], operation)
                        recursive_all_ten(results, numbers_copy + [temp_1], counting_only=counting_only)
                        temp_2 = apply_operation(option[1], option[0], operation)
                        recursive_all_ten(results, numbers_copy + [temp_2], counting_only=counting_only)

                else:
                    # in the case of & opeartion we are only allowed to concatenate original numbers
                    ## TO-DO: improve this to maybe allow 3 digits to get concatened (in the original
                    ## game the only allowed numbers are 1-9 so this would not result in any valid 
                    ## equiation but would be interesting to try.)
                    if len(option[0][1]) > 1 or len(option[1][1]) > 1:
                        pass
                    elif option[0][0] == option[0][1]:
                        # if both values are the same then this also becomes commutative
                        temp = apply_operation(option[0], option[1], operation)
                        recursive_all_ten(results, numbers_copy + [temp], counting_only=counting_only)
                    else:
                        temp_1 = apply_operation(option[0], option[1], operation)
                        recursive_all_ten(results, numbers_copy + [temp_1], counting_only=counting_only)
                        temp_2 = apply_operation(option[1], option[0], operation)
                        recursive_all_ten(results, numbers_copy + [temp_2], counting_only=counting_only)

    return results

## RUN

In [14]:
# numbers = [2, 2, 7, 9]
numbers = [2, 2, 7, 7]
results = {1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: []}

numbers = prepare_numbers(numbers)
results = recursive_all_ten(results, numbers)
print(results)

{1: ['((2 + 7) ÷ (2 + 7))', '(7 ÷ ((2 + 7) - 2))', '(2 ÷ ((2 + 7) - 7))', '((2 - 7) ÷ (2 - 7))', '(7 ÷ (2 - (2 - 7)))', '(2 ÷ (7 + (2 - 7)))', '((7 - 2) ÷ (7 - 2))', '(7 ÷ (2 + (7 - 2)))', '(2 ÷ (7 - (7 - 2)))', '((2 ÷ 7) ÷ (2 ÷ 7))', '((2 ÷ 7) * (7 ÷ 2))', '(2 ÷ (7 * (2 ÷ 7)))', '(7 ÷ (2 ÷ (2 ÷ 7)))', '(7 * ((2 ÷ 7) ÷ 2))', '((7 ÷ 2) * (2 ÷ 7))', '((7 ÷ 2) ÷ (7 ÷ 2))', '(2 ÷ (7 ÷ (7 ÷ 2)))', '(2 * ((7 ÷ 2) ÷ 7))', '(7 ÷ (2 * (7 ÷ 2)))', '((2 * 7) ÷ (2 * 7))', '(2 * (7 ÷ (2 * 7)))', '(2 ÷ ((2 * 7) ÷ 7))', '(7 * (2 ÷ (2 * 7)))', '(7 ÷ ((2 * 7) ÷ 2))', '(27 ÷ 27)', '(72 ÷ 72)', '(7 ÷ (7 + (2 - 2)))', '(7 ÷ (7 - (2 - 2)))', '((2 - 2) + (7 ÷ 7))', '((7 ÷ 7) - (2 - 2))', '((7 + (2 ÷ 2)) - 7)', '(7 - (7 - (2 ÷ 2)))', '(7 + ((2 ÷ 2) - 7))', '(7 ÷ (7 ÷ (2 ÷ 2)))', '(7 * ((2 ÷ 2) ÷ 7))', '(7 ÷ (7 * (2 ÷ 2)))', '((2 ÷ 2) + (7 - 7))', '((2 ÷ 2) - (7 - 7))', '((2 ÷ 2) ÷ (7 ÷ 7))', '((2 ÷ 2) * (7 ÷ 7))', '((7 - 7) + (2 ÷ 2))', '(2 ÷ (2 + (7 - 7)))', '(2 ÷ (2 - (7 - 7)))', '((7 ÷ 7) + (2 - 2))', '((

## ANALYSIS

In [15]:
print(f"target   # of solutions")
for key, value in results.items():
    print(f"{' '*(5-(key//10))}{key}   {len(value)}")

target   # of solutions
     1   52
     2   8
     3   6
     4   64
     5   9
     6   4
     7   6
     8   5
     9   7
    10   16


In [16]:
print(f"target   random solutions")
for key, value in results.items():
    print(f"{' '*(5-(key//10))}{key}   {value[0]}")

target   random solutions
     1   ((2 + 7) ÷ (2 + 7))
     2   ((2 ÷ 2) + (7 ÷ 7))
     3   (27 ÷ (2 + 7))
     4   ((2 + 7) + (2 - 7))
     5   ((2 * 7) - (2 + 7))
     6   ((7 - (2 - 7)) ÷ 2)
     7   ((7 ÷ 2) + (7 ÷ 2))
     8   (72 ÷ (2 + 7))
     9   ((2 - 7) + (2 * 7))
    10   ((7 - 2) - (2 - 7))


In [17]:
print(f"target   # of solutions   all solutions")
for key, value in results.items():
    print(f"{' '*(5-(key//10))}{key}   {len(value):<14}   {value}")

target   # of solutions   all solutions
     1   52               ['((2 + 7) ÷ (2 + 7))', '(7 ÷ ((2 + 7) - 2))', '(2 ÷ ((2 + 7) - 7))', '((2 - 7) ÷ (2 - 7))', '(7 ÷ (2 - (2 - 7)))', '(2 ÷ (7 + (2 - 7)))', '((7 - 2) ÷ (7 - 2))', '(7 ÷ (2 + (7 - 2)))', '(2 ÷ (7 - (7 - 2)))', '((2 ÷ 7) ÷ (2 ÷ 7))', '((2 ÷ 7) * (7 ÷ 2))', '(2 ÷ (7 * (2 ÷ 7)))', '(7 ÷ (2 ÷ (2 ÷ 7)))', '(7 * ((2 ÷ 7) ÷ 2))', '((7 ÷ 2) * (2 ÷ 7))', '((7 ÷ 2) ÷ (7 ÷ 2))', '(2 ÷ (7 ÷ (7 ÷ 2)))', '(2 * ((7 ÷ 2) ÷ 7))', '(7 ÷ (2 * (7 ÷ 2)))', '((2 * 7) ÷ (2 * 7))', '(2 * (7 ÷ (2 * 7)))', '(2 ÷ ((2 * 7) ÷ 7))', '(7 * (2 ÷ (2 * 7)))', '(7 ÷ ((2 * 7) ÷ 2))', '(27 ÷ 27)', '(72 ÷ 72)', '(7 ÷ (7 + (2 - 2)))', '(7 ÷ (7 - (2 - 2)))', '((2 - 2) + (7 ÷ 7))', '((7 ÷ 7) - (2 - 2))', '((7 + (2 ÷ 2)) - 7)', '(7 - (7 - (2 ÷ 2)))', '(7 + ((2 ÷ 2) - 7))', '(7 ÷ (7 ÷ (2 ÷ 2)))', '(7 * ((2 ÷ 2) ÷ 7))', '(7 ÷ (7 * (2 ÷ 2)))', '((2 ÷ 2) + (7 - 7))', '((2 ÷ 2) - (7 - 7))', '((2 ÷ 2) ÷ (7 ÷ 7))', '((2 ÷ 2) * (7 ÷ 7))', '((7 - 7) + (2 ÷ 2))', '(2 ÷ (2 +

In [18]:
print(f"target   # of & solutions   all solutions")
for key, value in results.items():
    concat_solutions = [sum([solution.count(op) for op in ["*", "-", "+", "÷"]]) == 2 for solution in value]
    value = list(compress(value, concat_solutions))
    print(f"{' '*(5-(key//10))}{key}   {len(value):<16}   {value}")

target   # of & solutions   all solutions
     1   0                  []
     2   0                  []
     3   1                  ['(27 ÷ (2 + 7))']
     4   0                  []
     5   0                  []
     6   0                  []
     7   0                  []
     8   3                  ['(72 ÷ (2 + 7))', '((22 - 7) - 7)', '(22 - (7 + 7))']
     9   0                  []
    10   2                  ['((27 - 7) ÷ 2)', '((72 - 2) ÷ 7)']


# VALID PROBLEMS SEARCH

A brute force search of what starting numbers are valid (or better said non-valid) - can produce all ten target numbers with only these operations: +, -, *, ÷ and &.

In [ ]:
import os
from tqdm.notebook import tqdm

path = "non_valid_starts.txt"

if os.path.exists(path):
    print(f"{path} already exists... You already have the results.")
else:
    with open(path, "w") as text_file:
            text_file.write(f"Idx   Starting numbers   Targets w/o solutions\n") #16

    digits = [i for i in range(1, 10)]

    number_of_non_valid = 0

    for start in tqdm(list(combinations_with_replacement(digits, 4))):
        numbers_org = list(start)
        results = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0} # initiate results differe tly since we will only count 
                                                                                # the number of solutions

        numbers = prepare_numbers(numbers_org.copy())
        results = recursive_all_ten_v2(results, numbers, counting_only=True)

        if 0 in results.values():
            non_valid = list(compress(results.keys(), [v == 0 for v in results.values()]))
            with open(path, "a") as text_file:
                text_file.write(f"{number_of_non_valid:<3}   {', '.join(map(str, numbers_org)):<16}   {' '.join(map(str, non_valid))}\n")
            number_of_non_valid+=1

non_valid_starts.txt already exists... You already have the results.


## PROBLEMS WITH & AS THE ONLY SOLUTION

In [30]:
import os
from tqdm.notebook import tqdm

path = "concatenated_only.txt"

if os.path.exists(path):
    print(f"{path} already exists... You already have the results.")
else:
    with open(path, "w") as text_file:
            text_file.write(f"Idx   Starting numbers   Valid start   & only solutions\n")

    digits = [i for i in range(1, 10)]
    idx = 0

    for start in tqdm(list(combinations_with_replacement(digits, 4))):
        numbers_org = list(start)
        results = {1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: []}


        numbers = prepare_numbers(numbers_org.copy())
        results = recursive_all_ten(results, numbers)

        concat_only = {}
        valid = True

        for key, value in results.items():
            concat_solutions = [sum([solution.count(op) for op in ["*", "-", "+", "÷"]]) == 2 for solution in value]
            if len(value) == sum(concat_solutions) and sum(concat_solutions)>=1:
                concat_only[key] = value
            
            if len(value) == 0:
                valid = False

        if concat_only:
            with open(path, "a") as text_file:
                text_file.write(f"{idx:<3}   {', '.join(map(str, numbers_org)):<16}   {str(valid):<11}   {list(concat_only.keys())[0]} -> {list(concat_only.values())[0]}\n")
                for i in range(1, len(concat_only)):
                    text_file.write(f"{' '*39}{list(concat_only.keys())[i]} -> {list(concat_only.values())[i]}\n")
            idx+=1

  0%|          | 0/495 [00:00<?, ?it/s]

## PROBLEMS WITH LEAST SOLUTIONS

In [67]:
import os
from tqdm.notebook import tqdm

path = "least_solutions.txt"

if os.path.exists(path):
    print(f"{path} already exists... You already have the results.")
else:
    with open(path, "w") as text_file:
        text_file.write(f"Idx   Starting numbers   Sum of solutions   target   # of solutions   solutions\n")

    digits = [i for i in range(1, 10)]
    idx = 0

    results_dict = {}

    for start in tqdm(list(combinations_with_replacement(digits, 4))):
        numbers_org = list(start)
        results = {1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: []}

        numbers = prepare_numbers(numbers_org.copy())
        results = recursive_all_ten(results, numbers)

        number_of_solutions = np.array([len(results[key]) for key in results.keys()])

        if np.all(number_of_solutions>0):
             results_dict[str(numbers_org)[1:-1]] = (int(sum(number_of_solutions)), results)


    sorted_results = dict(sorted(results_dict.items(), key=lambda item: item[1][0]))
    with open(path, "a") as text_file:
        for i, key in enumerate(sorted_results.keys()):
            text_file.write(f"{i:<5} {key:<18} {sorted_results[key][0]:<18} {list(sorted_results[key][1].keys())[0]:<8} {len(list(sorted_results[key][1].values())[0]):<16} {list(sorted_results[key][1].values())[0]}\n")
            for keykey in list(sorted_results[key][1].keys())[1:]:
                text_file.write(" "*44 + f"{keykey:<8} {len(sorted_results[key][1][keykey]):<16} {sorted_results[key][1][keykey]}\n")
            text_file.write("━"*100 + "\n")

  0%|          | 0/495 [00:00<?, ?it/s]